In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [55]:
%run scripts/helper.py
%run scripts/model_train_plus_test.py

In [73]:
crowd_train = load_file('./data/train.csv', None)
crowd_test = load_file('./data/test.csv', None)

In [74]:
target = crowd_train.median_relevance.values

In [58]:
# train_index, test_index = ssSplit(target, train_size=8000, random_state=44)

In [87]:
# X = tweak_text(crowd_train.iloc[train_index])
# Xv = tweak_text(crowd_train.iloc[test_index])
X = tweak_text(crowd_train)
Xv = tweak_text(crowd_test)

In [88]:
# y = target[train_index]
# yv = target[test_index]
y = target

In [89]:
Xfitted, tfv = TFIDF(X, None)

In [51]:
svm_non_linear, svd, scl = build_non_linear_model(Xfitted, y)

In [52]:
options = {
    'tfv': tfv,
    'svd': svd,
    'scl': scl
}

preds = make_predictions(svm_non_linear, options, Xv)

In [53]:
print 'Kappa score on validation set %0.4f ' %(quadratic_weighted_kappa(yv, preds))

Kappa score on validation set 0.5690 


In [14]:
from sklearn.metrics import confusion_matrix

In [15]:
confusion_matrix(yv, preds)

array([[ 40,  15,  10,  12],
       [ 20,  55,  16,  57],
       [ 11,  25,  28, 110],
       [ 13,  22,  33, 549]])

In [106]:
# Analyze the mistakes of this model
mistakes_idx = [test_index[i] for i in range(len(preds)) if yv[i] != preds[i]]
incorrect_preds = [preds[i] for i in range(len(preds)) if yv[i] != preds[i]]

In [107]:
mistakes = crowd_train.iloc[mistakes_idx]

In [108]:
mistakes['pred'] = incorrect_preds

In [109]:
mistakes.shape

(371, 7)

In [110]:
mistakes.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,pred
6558,21149,ps2 controller usb,INSTEN For Xbox 360 Wireless Game Controller U...,USB Charging Cable compatible with Microsoft X...,1,0.000,2
5801,18673,converse high tops,Converse CT Ox Youth Girls Size 3 Bronze Texti...,"The Converse brand of athletic shoes, which no...",2,0.471,4
10099,32458,wii microphone,Insten For XBOX 360 SLIM Black Headset With Mi...,"Headset compatible with Microsoft Xbox 360, Bl...",1,0.000,2
1465,4733,high heels shoes,New Womens Ankle Strap Platform Shoes Hidden W...,,3,0.000,1
6859,22151,light bulb,LB001 Retro Light Bulb,The standard since the beginning of the centur...,3,0.000,4


In [157]:
for i in range(5, 10, 1):
    print 'Query:  %s ' %(mistakes.iloc[i].query)
    print 'Title: %s ' %(mistakes.iloc[i].product_title)
    print 'Description: %s ' %(mistakes.iloc[i].product_description)
    print '########################################'
    print 'Relevance %d ' %(mistakes.iloc[i].median_relevance)
    print 'My Prediction %d ' %(mistakes.iloc[i].pred)
    print '\n\n'

Query:  samsonite 
Title: Samsonite Winfield 2 Fashion 24 
Description: Travel in style with this 24-inch upright suitcase in orange or charcoal. Wheels and a telescopic handle add convenient maneuverability. 
########################################
Relevance 4 
My Prediction 2 



Query:  videogames ps4 
Title: Metal Gear Solid V: Ground Zeroes - PlayStation 4 
Description:  
########################################
Relevance 3 
My Prediction 4 



Query:  flower bulbs 
Title: TotalGreen 72405500 Rembrandt Tulip Flower Bulb-13PK HEMISPHERE TULIPS 
Description: 13pk hemisphere tulips. Rembrandt mixed colors of tulips. Size: 11 to 12 cm. Packed 13 bulbs per bag. You may be required by your State Department of Agriculture to apply for a nursery stock dealer license. This can be verified by contacting your State Department of Agriculture or your local County Extension office. Do it Best Corp will try to stay in stock on this SKU throughout the season; however, due to the weather and the 

In [177]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [178]:
def query_terms_in_results(train):
    query_terms = train['query'].split(' ')
    response = train['product_title'] + ' ' + train['product_description']
    keyword = False
    
    for q in query_terms:
        if q not in stop:
            keyword = True
            if response.lower().find(q) == -1:
                return 0
    
    if keyword == False:
        return 0
    else:
        return 1
    
is_query_in_response = crowd_train.apply(query_terms_in_results, axis=1)

In [179]:
feature_query_in_response = pd.DataFrame({'is_query_in_response': is_query_in_response,
              'median_relevance': crowd_train.median_relevance})

In [180]:
feature_query_in_response.corr()

,is_query_in_response,median_relevance
is_query_in_response,1.00000,0.34641
median_relevance,0.34641,1.00000


In [154]:
feature_query_in_response.groupby(['is_query_in_response', 'median_relevance']).size()

is_query_in_response  median_relevance
0                     1                    614
                      2                    882
                      3                    780
                      4                   1633
1                     1                    160
                      2                    594
                      3                    957
                      4                   4538
dtype: int64

In [148]:
# lets find out how many query terms found in response
def count_query_terms_in_response(train):
    query_terms = train['query'].split(' ')
    response = train['product_title'] + ' ' + train['product_description']
    
    count = 0
    for q in query_terms:
        if response.lower().find(q) != -1:
            count += 1
    return count

num_query_terms_in_response = crowd_train.apply(count_query_terms_in_response, axis=1)

In [149]:
feature_num_query_in_response = pd.DataFrame({'num_query_in_response': num_query_terms_in_response,
              'median_relevance': crowd_train.median_relevance})

In [150]:
feature_num_query_in_response.corr()

,median_relevance,num_query_in_response
median_relevance,1.000000,0.250085
num_query_in_response,0.250085,1.000000


In [156]:
feature_num_query_in_response.groupby(['num_query_in_response', 'median_relevance']).size()

num_query_in_response  median_relevance
0                      1                    176
                       2                    148
                       3                    129
                       4                    210
1                      1                    323
                       2                    495
                       3                    512
                       4                   1295
2                      1                    232
                       2                    598
                       3                    711
                       4                   3144
3                      1                     37
                       2                    190
                       3                    304
                       4                   1076
4                      1                      6
                       2                     40
                       3                     72
                       4                    408


In [169]:
# lexical diversity
def word_diversity(train):
    response = train['product_title'] + ' ' + train['product_description']
    vocabulary = list(set([x.lower() for x in response.split(' ')]))
    
    return (len(vocabulary) * 1.) / len(response.split(' '))

word_diversity = crowd_train.apply(word_diversity, axis=1)

In [172]:
feature_word_diversity = pd.DataFrame({'word_diversity': word_diversity,
                                       'median_relevance': crowd_train.median_relevance})

In [173]:
feature_word_diversity.corr()

,median_relevance,word_diversity
median_relevance,1.000000,0.026877
word_diversity,0.026877,1.000000


In [174]:
# how big the response is
def response_length(train):
    response = train['product_title'] + ' ' + train['product_description']
    return len(response.split(' '))

query_response_length = crowd_train.apply(response_length, axis=1)

In [175]:
feature_response_length = pd.DataFrame({'response_length': query_response_length,
                                        'median_relevance': crowd_train.median_relevance})

In [176]:
feature_response_length.corr()

,median_relevance,response_length
median_relevance,1.000000,-0.022693
response_length,-0.022693,1.000000


### Stacked model with handcrafted features

In [90]:
%run query_features.py

In [91]:
svd = TruncatedSVD(n_components=200, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)

scl = StandardScaler(copy=True, with_mean=True, with_std=True)

clf = SVC(C=10.0, kernel='rbf', degree=3, 
        gamma=0.0, coef0=0.0, shrinking=True, probability=False, 
        tol=0.001, cache_size=200, class_weight=None, 
        verbose=False, max_iter=-1, random_state=None)

# keywords = keyword_counter(crowd_train.iloc[train_index])
keywords = keyword_counter(crowd_train)

In [92]:
features = stack([keywords, Xfitted])

In [93]:
from sklearn.pipeline import Pipeline

In [94]:
pipeline = Pipeline([('svd', svd), ('scl', scl), ('clf', clf)])

In [95]:
pipeline.fit(features, y)

Pipeline(steps=[('svd', TruncatedSVD(algorithm='randomized', n_components=200, n_iter=5,
       random_state=None, tol=0.0)), ('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [96]:
# keywords_test = keyword_counter(crowd_train.iloc[test_index])
keywords_test = keyword_counter(crowd_test)

In [97]:
Xtest = tfv.transform(Xv)

In [98]:
features_test = stack([keywords_test, Xtest])

In [99]:
preds_new_model = pipeline.predict(features_test)

In [72]:
#print 'Kappa score on validation set %0.4f ' %(quadratic_weighted_kappa(yv, preds_new_model))

Kappa score on validation set 0.6011 


### Load out best prediction thus far

In [100]:
best_score_df = pd.read_csv('./submissions/best_score.csv')

In [101]:
best_score = best_score_df.prediction

In [102]:
ensemble = (preds_new_model + best_score) / 2
ensemble_int = [int(score) for score in ensemble]

In [104]:
make_submission(crowd_test.id.values.astype(int), ensemble_int, 'addtitional_features_500.csv')